In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from itertools import islice

import requests
import json

In [2]:
from googletrans import Translator
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
import re, string, unicodedata
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import joblib

# 노래가사 SVM

- T5 요약이 끝난 노래가사 csv 파일 불러오기
- 각 노래 가사 text에서 감정 추출 진행

**가사 요약 결과 텍스트 전처리**

- `song` = `lyrics_summary_final.xlsx`
- 불필요한 `<pad>` 부분 제거
- 텍스트 전처리 함수에 적용

In [15]:
song = pd.read_excel('lyrics_summary_final.xlsx',index_col=0)
print(song.shape)
song.head()

(21027, 7)


,url,name,id,Artist,lyrics,lyrics_english,summary
0.0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Still With You,0eFMbKCRw8KByXyWBw8WO7,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...,<pad> if face again wanted see eyes and told I...
1.0,https://open.spotify.com/track/7eJMfftS33KTjuF...,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,Powfu,Don t stay awake for too long don t go to bed...,Don t stay awake for too long don t go to bed...,<pad> don t stay awake for too long don t go t...
2.0,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,Fujii Kaze,あんたのその歯に はさがった青さ粉に\nふれるべきか否かで少し悩んでる\n口にしない方がいい...,In the bluish powder that has exposed to your ...,<pad> what is it Before I said Then what Why d...
3.0,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,Fujii Kaze,さぁ羽のばして ここから\n捉われてばっか だったから\n行き詰った悦び手放す時は今\n心軽...,Now from here\nBecause it was just caught\nNow...,<pad> the night was blown by the cold wind Don...
4.0,https://open.spotify.com/track/57u1FaZlztApkcT...,YASASHISA,57u1FaZlztApkcT5x5XNuD,Fujii Kaze,今何を見ていた あなたの夢を見た\n優しさに殺られた あの人の木陰で\n今何を見ていた あな...,What your dreams were now having your dreams\n...,<pad> what your dreams were now having your dr...


In [10]:
def hapus_url(text):
    mention_pattern = r'@[\w]+'
    cleaned_text = re.sub(mention_pattern, '', text)
    return re.sub(r'http\S+','', cleaned_text)
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-Z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
def stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text=' '.join([ps.stem(word) for word in text.split()])
    return text

def final_clean(text):
    final_text= []
    for i in text.split():
        if i.strip().lower() not in sw and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def clean(text):
    text = hapus_url(text)
    text = remove_special_characters(text, remove_digits=True)
    text = stemmer(text)
    text = final_clean(text)
    return text

In [9]:
def replace_pad(x):
    return x.replace('<pad>',"")

song['summary'] = song['summary'].apply(replace_pad)
song['summary'][1]

' don t stay awake for too long don t go to bed I ll make a cup of coffee for your head It ll get you up and going out of bed Don t stay awake for too long don t go to bed I ll make a cup of coffee for your head It ll get you up and going out of bed Don t stay awake for too long don t go to bed I ll make a cup of coffee for your head It ll get you up and going out of bed Don </s>'

In [12]:
def clean_summary(x):
    return clean(x)

In [16]:
nltk.download('stopwords')
sw = stopwords.words("english")
translator = Translator()
song['summary_text'] = song['summary'].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\82109\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
song.head()

,url,name,id,Artist,lyrics,lyrics_english,summary,summary_text
0.0,https://open.spotify.com/track/0eFMbKCRw8KByXy...,Still With You,0eFMbKCRw8KByXyWBw8WO7,Jung Kook,날 스치는 그대의 옅은 그 목소리\n내 이름을 한 번만 더 불러주세요\n얼어버린 노...,Your pale voice\nPlease call my name once more...,<pad> if face again wanted see eyes and told I...,pad face want see eye told ecstat memori even ...
1.0,https://open.spotify.com/track/7eJMfftS33KTjuF...,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,Powfu,Don t stay awake for too long don t go to bed...,Don t stay awake for too long don t go to bed...,<pad> don t stay awake for too long don t go t...,pad stay awak long go bed make cup coffe head ...
2.0,https://open.spotify.com/track/2CbGuO0LtVvbh3u...,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,Fujii Kaze,あんたのその歯に はさがった青さ粉に\nふれるべきか否かで少し悩んでる\n口にしない方がいい...,In the bluish powder that has exposed to your ...,<pad> what is it Before I said Then what Why d...,pad befor said whi didnt ask anyth face w swor...
3.0,https://open.spotify.com/track/1JRNrJKTQhfvxII...,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,Fujii Kaze,さぁ羽のばして ここから\n捉われてばっか だったから\n行き詰った悦び手放す時は今\n心軽...,Now from here\nBecause it was just caught\nNow...,<pad> the night was blown by the cold wind Don...,pad night wa blown cold wind dont wander blown...
4.0,https://open.spotify.com/track/57u1FaZlztApkcT...,YASASHISA,57u1FaZlztApkcT5x5XNuD,Fujii Kaze,今何を見ていた あなたの夢を見た\n優しさに殺られた あの人の木陰で\n今何を見ていた あな...,What your dreams were now having your dreams\n...,<pad> what your dreams were now having your dr...,pad dream dream shade person wa kill kind drea...


**노래 가사 text에 SVM 모델 적용**

- 노래 가사 text 감정 벡터 추출 결과 = `lyrics_sentiment.xlsx`
- SVM 모델이 트위터 데이터로 학습했기 때문에 학습 당시 사용했던 input 형태와 동일한 형태로 현재 가사 text input을 변형해야함
- tf-idf 모델을 트위터 데이터로 fit_transform, 가사 text 데이터에 transform

In [18]:
df = pd.read_csv('tweet_data_agumentation.csv', index_col = 0)
df['sentiment'].value_counts()

6     14977
4      8254
7      7494
11     6146
8      5539
1      5055
9      5020
0      3007
2      3003
10     2960
12     1724
Name: sentiment, dtype: int64

In [19]:
model = joblib.load('SVM.pkl')
model

SVC(probability=True)

In [20]:
# df로 tf-idf 학습
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit_transform(df["content"])

# 학습한 tf-idf 모델을 가사 text에 동일하게 적용
text2 = tfidf_vect.transform(song['summary_text'])
pred = model.predict_proba(text2)

In [20]:
sentiment = pd.DataFrame(pred,columns=['empty','sadness','enthusiasm','worry','love','fun','hate','happiness','boredom','relief','anger'])
sentiment.insert(0,'id',song['id'])
sentiment.insert(0,'name',song['name'])
sentiment

,name,id,empty,sadness,enthusiasm,worry,love,fun,hate,happiness,boredom,relief,anger
0,Still With You,0eFMbKCRw8KByXyWBw8WO7,0.016461,0.265974,0.067839,0.144616,0.116033,0.206217,0.015079,0.134595,0.002582,0.028249,0.002356
1,death bed (coffee for your head),7eJMfftS33KTjuF7lTsMCx,0.122187,0.059084,0.039108,0.113228,0.024429,0.023013,0.015683,0.212476,0.004940,0.384384,0.001468
2,Nan-Nan,2CbGuO0LtVvbh3umN3mDwM,0.005816,0.248961,0.003242,0.483580,0.044740,0.008447,0.022421,0.152159,0.000405,0.029745,0.000483
3,Mo-Eh-Wa,1JRNrJKTQhfvxII8kkIcAQ,0.077518,0.249271,0.011364,0.406069,0.011648,0.038887,0.005062,0.123563,0.002235,0.072786,0.001598
4,YASASHISA,57u1FaZlztApkcT5x5XNuD,0.001735,0.235752,0.001410,0.463885,0.142529,0.009176,0.010140,0.120540,0.000858,0.013238,0.000737
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21022,Turn Me Up - ViP Mix,5ebVfX8yQhlRijjQBUaHfp,0.196301,0.208191,0.020845,0.145600,0.147506,0.123824,0.009645,0.123349,0.006121,0.013332,0.005287
21023,Miserable Man,74bZgwiYa25ZP3isFCm8OK,0.005901,0.337508,0.010246,0.586264,0.025339,0.004074,0.013446,0.013156,0.000391,0.003570,0.000107
21024,Out Of Love,3YQjy4sRBnFIXyBYBIANdt,0.003902,0.093651,0.004717,0.440373,0.111086,0.032052,0.003084,0.305699,0.000648,0.004264,0.000522
21025,Murder in the City,74xCxQy50bkpvOJm5MgD5P,0.011245,0.162807,0.006966,0.456847,0.082303,0.040452,0.002504,0.231635,0.000722,0.004143,0.000376


In [21]:
sentiment.to_excel('lyrics_sentiment.xlsx')